# Based on Context Window (Co-occurrence)

In [1]:
import pandas as pd
from collections import defaultdict

## Read text

In [2]:
with open('review.sorted.uniq.refined.tsv.text.tok') as f:
    lines = [l.strip() for l in f.read().splitlines() if l.strip()]

## Define methods

In [3]:
def get_term_frequency(document):
    term_freq = {}
    
    words = document.split()

    for w in words:
        term_freq[w] = 1 + (0 if term_freq.get(w) is None else term_freq[w])

    return term_freq

In [4]:
def get_context_counts(lines, vocab, w_size=2):
    context_cnt = defaultdict(int)
    
    for line in lines:
        words = line.split()
        
        for i, w in enumerate(words):
            if w in vocab:
                for c in words[i - w_size:i + w_size]:
                    if w != c:
                        context_cnt[(w, c)] += 1
            
    return context_cnt

In [5]:
def get_co_occurrence_df(context_cnt, vocab):
    data = []
    
    for word1 in vocab:
        row = []
        
        for word2 in vocab:
            try:
                count = context_cnt[(word1, word2)]
            except KeyError:
                count = 0
            row.append(count)
            
        data.append(row)
    
    return pd.DataFrame(data, index=vocab, columns=vocab)

## Call methods

Count frequency of each word.

In [6]:
term_freq = pd.Series(
    get_term_frequency(' '.join(lines))
).sort_values(ascending=False)

term_freq

.         86303
고         49631
이         44952
하         42916
좋         34589
          ...  
두드려서          1
gvudsf        1
ㅕㅅ            1
나후            1
녹물            1
Length: 30084, dtype: int64

In [7]:
vector_size = 800

Because most of words are seen only few times, we don't need all of them.<br>
Thus, we can truncate low-frequency words.

In [8]:
term_freq.index[:vector_size]

Index(['.', '고', '이', '하', '좋', '네요', '도', '에', '는', '가',
       ...
       '한쪽', '엄마', '가을', 'ㅁ', '국산', '요청', '마', '보풀', '세일', '싸구려'],
      dtype='object', length=800)

Now, we count with context windowing based on truncated dictionary.

In [9]:
context_cnt = pd.Series(
    get_context_counts(
        lines,
        term_freq.index[:vector_size],
        w_size=4
    )
)

context_cnt

라고  비지떡     31
    ".       1
    200      2
    ml       5
    판매      16
          ... 
았   ㅍ        1
감사  ㅍㅍ       2
고   수수     106
수고  수수     212
    고수       3
Length: 1047278, dtype: int64

Build Pandas DataFrame with given context-windowing result.

In [10]:
df = get_co_occurrence_df(context_cnt, term_freq.index[:vector_size])

df

,.,고,이,하,좋,네요,도,에,는,가,...,한쪽,엄마,가을,ㅁ,국산,요청,마,보풀,세일,싸구려
.,0,9200,11111,9760,8211,14149,7929,6816,5063,5321,...,41,45,63,14,44,68,30,61,35,85
고,8762,0,4682,13988,7102,3398,15105,3634,3142,2426,...,62,15,26,17,23,30,47,41,18,30
이,10028,4767,0,3377,2854,6044,2169,3326,4554,2224,...,140,18,35,12,101,14,21,128,51,42
하,9051,13688,3625,0,3654,5694,4859,2738,5121,1925,...,19,40,9,20,13,63,30,18,60,15
좋,8096,8274,2990,3881,0,5551,5104,1636,1194,1210,...,0,28,33,23,15,0,9,11,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
요청,68,29,18,59,1,11,17,33,4,14,...,0,0,0,0,0,0,0,0,0,0
마,32,51,19,35,8,37,12,14,10,13,...,0,0,0,0,0,0,0,0,0,0
보풀,54,46,124,23,14,36,26,38,22,6,...,0,0,0,0,0,0,0,0,0,0
세일,37,25,22,50,12,13,10,22,18,5,...,0,0,0,0,22,0,0,0,0,0


In [11]:
df.values.shape

(800, 800)

Now, we have co-occurrence vectors, but you can see that there are still many sparse vectors in the DataFrame.

homework: Take a PCA to reduce the dimension of the vectors.

# Similarity Measures

## Define metrics

In [12]:
import torch

$$
\text{d}_{\text{L1}}(w,v)=\sum_{i=1}^d{|w_i-v_i|},\text{ where }w,v\in\mathbb{R}^d.
$$

In [13]:
def get_l1_distance(x1, x2):
    return ((x1 - x2).abs()).sum()

$$
\text{d}_{\text{L2}}(w,v)=\sqrt{\sum_{i=1}^d{(w_i-v_i)^2}},\text{ where }w,v\in\mathbb{R}^d.
$$

In [14]:
def get_l2_distance(x1, x2):
    return ((x1 - x2)**2).sum()**.5

$$
d_{\infty}(w,v)=\max(|w_1-v_1|,|w_2-v_2|,\cdots,|w_d-v_d|),\text{ where }w,v\in\mathbb{R}^d
$$

In [15]:
def get_infinity_distance(x1, x2):
    return ((x1 - x2).abs()).max()

$$
\begin{aligned}
\text{sim}_{\text{cos}}(w,v)&=\overbrace{\frac{w\cdot v}{|w||v|}}^{\text{dot product}}
=\overbrace{\frac{w}{|w|}}^{\text{unit vector}}\cdot\frac{v}{|v|} \\
&=\frac{\sum_{i=1}^{d}{w_iv_i}}{\sqrt{\sum_{i=1}^d{w_i^2}}\sqrt{\sum_{i=1}^d{v_i^2}}} \\
\text{where }&w,v\in\mathbb{R}^d
\end{aligned}
$$

In [16]:
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5 + 1e-10)

$$
\begin{aligned}
\text{sim}_{\text{jaccard}}(w,v)&=\frac{|w \cap v|}{|w \cup v|} \\
&=\frac{|w \cap v|}{|w|+|v|-|w \cap v|} \\
&\approx\frac{\sum_{i=1}^d{\min(w_i,v_i)}}{\sum_{i=1}^d{\max(w_i,v_i)}} \\
\text{where }&w,v\in\mathbb{R}^d.
\end{aligned}
$$

Because above methods are defined with same interface, we can switch it if we needed.

In [17]:
def get_nearest(query, dataframe, metric, top_k, ascending=True):
    vector = torch.from_numpy(dataframe.loc[query].values).float()
    distances = dataframe.apply(
        lambda x: metric(vector, torch.from_numpy(x.values).float()),
        axis=1,
    )
    top_distances = distances.sort_values(ascending=ascending)[:top_k]

    print(', '.join([f'{k} ({v:.1f})' for k, v in top_distances.items()]))

## Show nearest neighbor of given word for each metric.

In [18]:
print('L1 distance:')
get_nearest('반품', df, get_l1_distance, 10)
print('\nL2 distance:')
get_nearest('반품', df, get_l2_distance, 10)
print('\nInfinity distance:')
get_nearest('반품', df, get_infinity_distance, 10)
print('\nCosine similarity:')
get_nearest('반품', df, get_cosine_similarity, 10, ascending=False)

L1 distance:
반품 (0.0), 교환 (7246.0), ㅠㅠ (8674.0), ㅠ (8832.0), 말 (9029.0), 다시 (9094.0), 다고 (9220.0), 확인 (9378.0), 그리고 (9467.0), 못 (9480.0)

L2 distance:
반품 (0.0), 교환 (984.0), 다고 (1076.1), 깔끔 (1125.0), 다시 (1207.6), 확인 (1213.0), 싶 (1229.0), 여 (1279.8), 포장 (1285.4), 긴 (1302.4)

Infinity distance:
반품 (0.0), 다고 (456.0), 를 (458.0), 깔끔 (461.0), 여 (510.0), 긴 (522.0), 로 (534.0), 확인 (547.0), 교환 (564.0), 싶 (565.0)

Cosine similarity:
반품 (1.0), 교환 (0.9), 조립 (0.9), 환불 (0.9), 사용 (0.9), 확인 (0.9), 작업 (0.9), 기 (0.8), 설치 (0.8), 다고 (0.8)
